# Векторная база данных

In [1]:
import os
from pathlib import Path
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb import Client
import chromadb
from langchain.schema import Document

def load_documents(data_dir: str) -> List[Dict]:
    """Создаем чанки с помощью RecursiveCharacterTextSplitter"""
    documents = []
    splitter = splitter = RecursiveCharacterTextSplitter(
                            chunk_size=700,               # Размер чанка 
                            chunk_overlap=50,            # Размер перекрытия чанков
                            separators=["\n\n", "\n", ".", " ", ""],  # Приоритет разбиения: абзац → строка → предложение → пробел
                          )

    for root, _, files in os.walk(data_dir):
        # Пропускаем системные папки
        if '.ipynb_checkpoints' in root:
            continue

        for file in files:
            if file.endswith(".txt") and not file.startswith('.') and '.ipynb_checkpoints' not in file:
                full_path = os.path.join(root, file)

                with open(full_path, "r", encoding="utf-8") as f:
                    text = f.read()

                country = Path(root).name
                city = Path(file).stem

                chunks = splitter.split_text(text)

                for chunk in chunks:
                    documents.append(
                        Document(
                            page_content=f"search_document: {chunk.strip()}",
                            metadata={
                                "country": country,
                                "city": city
                            }
                        )
                    )

    return documents

In [2]:
documents = load_documents("data")

In [3]:
documents[0]

Document(metadata={'country': 'Германия', 'city': 'Берлин'}, page_content='search_document: БЕРЛИН')

In [4]:
documents[1]

Document(metadata={'country': 'Германия', 'city': 'Берлин'}, page_content='search_document: ### Достопримечательности\n\nБерлинская стена .East Side Gallery -музей под открытым небом.Остатки Берлинской стены протяженностью 1316 метров, на которой нарисовано около 106 картин авторами из разных стран мира после падения стены в период с 1989 по 1990 год.Длина стены составляла около 155 км (43,1 км приходились непосредственно на город), а высота достигала 4,1 м. Сейчас ее обломки успешно продаются в виде сувениров. Находится вдоль Шпрее на улице Мюленштрассе,между Ostbahnhof и мостом Обербаум.Добраться можно пешком от Остбанхофа или пешком от U-Bahn Варшауэр Штрассе в сторону Обербаумбрюке до Mühlenstraße.')

## Загружаем SOTA эмбеддер для retrieval задачи

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Загружаем эмбеддер FRIDA
embedding = HuggingFaceEmbeddings(
    model_name="ai-forever/FRIDA",
    encode_kwargs={"normalize_embeddings": True}
)

C:\Users\kiril\AppData\Local\Temp\ipykernel_11528\2014946616.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


## Создаем векторную БД

In [9]:
from langchain.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="chroma_storage"
)

### Пример поиска

In [7]:
query = "Что посетить в питере?"
final_query = f"search_query: {query}"

results = chroma_db.similarity_search(final_query, k=2)

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 40)

search_document: САНКТ-ПЕТЕРБУРГ

### Достопримечательности

В Петербурге есть множество памятников архитектуры, скульптуры, различных мемориалов, музеев, театров разного направления. Здесь также представлены интересные сооружения на стыке инженерной мысли и искусства, такие как всемирно известные петербургские мосты.

Исторический центр Санкт-Петербурга включён в список всемирного наследия ЮНЕСКО.

Для удобства конкретные достопримечательности описываются в статьях по районам города. Наиболее интересны в этом отношении историческая часть города — его центр , Васильевский остров и Петроградская сторона .

### Чем заняться
{'city': 'Санкт-Петербург', 'country': 'Россия'}
----------------------------------------
search_document: ### Чем заняться

Альтернативный способ изучить город и узнать его изнутри - гулять и общаться с местными жителями и принимать участие в локальных мероприятиях. Люди, которые живут в Питере многие годы, хотели бы рассказать путешественникам свои истории, открыть 